# Logging Tracing Concept for Email Workflow

As the production system is not ready yet, and even if it would be there is not enough time to gather enough test data
to validate the concept necessary for demonstration.

To properly validate the concept the demo has to check the requirements below:
* simulate every component existing in future production system
* simulate errors / full failures in specific components to validate concept
* produce large amounts of realistic test data in a short amount of time

In [1]:
import sys
import random
import datetime

import logging
from logging import config

In [25]:
case_format = "CASE::%(case)s::%(levelname)s::%(message)s"
debug_format = "DEBUG::%(component)s::%(asctime)s::%(levelname)s::%(message)s"
debug_date_format = "%Y-%m-%d %H:%M:%S"

def setup_logger(name):
    level = logging.DEBUG
    config.dictConfig({
        'version': 1,
        'disable_existing_loggers': True,
        'formatters': {
            'debug': {'format': debug_format, 'datefmt': debug_date_format},
            'case': {'format': case_format}
        },
        'handlers': {
            'debug_console': {
                'class': 'logging.StreamHandler',
                'level': level,
                'formatter': 'debug',
                'stream': 'ext://sys.stdout'
            },
            'case_console': {
                'class': 'logging.StreamHandler',
                'level': level,
                'formatter': 'case',
                'stream': 'ext://sys.stdout'
            },
            'debug_file': {
                'class': 'logging.FileHandler',
                'level': level,
                'formatter': 'debug',
                'filename': './log/debug.log',
                'filemode': 'w'
            },
            'case_file': {
                'class': 'logging.FileHandler',
                'level': level,
                'formatter': 'case',
                'filename': './log/case.log',
                'filemode': 'w'
            },
             'nice_weather_case_file': {
                'class': 'logging.FileHandler',
                'level': level,
                'formatter': 'case',
                'filename': './log/nice_weather_case.log',
                'filemode': 'w'
            },
             'long_correlator_case_file': {
                'class': 'logging.FileHandler',
                'level': level,
                'formatter': 'case',
                'filename': './log/long_correlator_case.log',
                'filemode': 'w'
            },
        },
        'loggers': {
            'debug': {
                'level': level,
                'handlers': ['debug_console', 'debug_file']
            },
            'case': {
                'level': level,
                'handlers': ['case_console', 'case_file']
            },
             'nice_weather_case': {
                'level': level,
                'handlers': ['case_console', 'nice_weather_case_file']
            },
            'long_correlator_case': {
                'level': level,
                'handlers': ['case_console', 'long_correlator_case_file']
            }
        }
    })
    return logging.getLogger(name)

debugLogger = setup_logger("debug")
caseLogger = setup_logger("case")

In [26]:
class LoggerWrapper:
    def __init__(self, logger, logger_props):
        self._logger_props = logger_props
        self._logger = logger

    def __getattr__(self, attr):
        local_props = self._logger_props
        def wrapped(message):
            return getattr(self._logger, attr)(message, extra=local_props)
        return wrapped

In [27]:
debugLogger.info("test debug logger", extra={'component':"Comp1"})
caseLogger.info("test case logger", extra={'case':"Case1"})

DEBUG::Comp1::2021-01-05 11:37:11::INFO::test debug logger
CASE::Case1::INFO::test case logger


In [28]:
class CaseCapsule:
    def __init__(self, id: str, logger: logging.Logger=caseLogger):
        # TODO
        self._id = id
        self._last_seen = datetime.datetime.today()
        self._counter = 1
        self._log = logger

    @property
    def id(self):
        return self._id

    @property
    def last_seen(self):
        return self._last_seen

    @property
    def counter(self):
        return self._counter

    def add_seconds(self, seconds):
        self._counter += 1
        timedelta = datetime.timedelta(seconds=seconds)
        self._last_seen += timedelta

    @property
    def log(self):
        return LoggerWrapper(self._log, {'case':str(self)})

    def __str__(self):
        return f"id {self._id}::{self._last_seen}::count {self._counter}"

In [29]:
case = CaseCapsule("42")
print(case)
case.add_seconds(100)
print(case)

id 42::2021-01-05 11:37:18.415389::count 1
id 42::2021-01-05 11:38:58.415389::count 2


In [30]:
newLogger = LoggerWrapper(caseLogger, {'case': 'hello_world'})

newLogger.debug('test log')

CASE::hello_world::DEBUG::test log


In [31]:
class Component:
    def __init__(self, name, pre_sub_function, post_sub_function, debug_logger=debugLogger, sub_component=None):
        self._name = name
        self._pre_sub_function = pre_sub_function
        self._post_sub_function = post_sub_function
        self._sub_component = sub_component
        self._debug_logger = LoggerWrapper(debug_logger, {'component':name})

    def exec(self, case: CaseCapsule):
        self._debug_logger.debug('before calling pre sub functions')
        self._pre_sub_function(case, self._debug_logger, self._name)
        self._debug_logger.debug('after calling pre sub functions')
        
        if self._sub_component:
            self._debug_logger.debug('before calling sub component')
            self._sub_component.exec(case)
            self._debug_logger.debug('after calling sub component')
            
        self._debug_logger.debug('before calling post sub functions')
        self._post_sub_function(case, self._debug_logger, self._name)
        self._debug_logger.debug('after calling post sub functions')

In [32]:
def test_pre_fun(case:CaseCapsule, logger: logging.Logger, name):
    logger.debug('some demo pre function')
    case.add_seconds(2)
    case.log.debug(f'executed pre of component {name}')

def test_post_fun(case:CaseCapsule, logger: logging.Logger, name):
    logger.debug('some demo post function')
    case.add_seconds(2)
    case.log.debug(f'executed post of component {name}')

comp = Component("TestComp", test_pre_fun, test_post_fun)

case = CaseCapsule("1")

comp.exec(case)

DEBUG::TestComp::2021-01-05 11:37:26::DEBUG::before calling pre sub functions
DEBUG::TestComp::2021-01-05 11:37:26::DEBUG::some demo pre function
CASE::id 1::2021-01-05 11:37:28.787043::count 2::DEBUG::executed pre of component TestComp
DEBUG::TestComp::2021-01-05 11:37:26::DEBUG::after calling pre sub functions
DEBUG::TestComp::2021-01-05 11:37:26::DEBUG::before calling post sub functions
DEBUG::TestComp::2021-01-05 11:37:26::DEBUG::some demo post function
CASE::id 1::2021-01-05 11:37:30.787043::count 3::DEBUG::executed post of component TestComp
DEBUG::TestComp::2021-01-05 11:37:26::DEBUG::after calling post sub functions


In [33]:
def simulate_timeout(ceil=10, likelihood=0.9):
    # decide if timeout should be added
    is_timeout = random.random()

    if is_timeout > likelihood:
        # add timeout between range of ceil
        return random.randrange(ceil*10)/10
    
    return 0.1

print(case)
case.add_seconds(simulate_timeout(10, 0.6))
print(case)

id 1::2021-01-05 11:37:30.787043::count 3
id 1::2021-01-05 11:37:30.887043::count 4


# Define simulation scenarios



## Nice weather case simulation
Nice weather case scenario models the process optimal response time and throughput

In [34]:
def nice_weather_pre_function(case:CaseCapsule, logger: logging.Logger, name):
    logger.debug('executing pre function')
    case.add_seconds(simulate_timeout(2, 0.9))
    case.log.debug(f'executed pre function of component {name}')

def nice_weather_post_function(case:CaseCapsule, logger: logging.Logger, name):
    logger.debug('executing post function')
    case.add_seconds(simulate_timeout(2, 0.9))
    case.log.debug(f'executed post function of component {name}')

notification_service = Component("notification_service", nice_weather_pre_function, nice_weather_post_function, debugLogger)
correlator = Component("correlator", nice_weather_pre_function, nice_weather_post_function, debugLogger, notification_service)
analytics = Component("analytics", nice_weather_pre_function, nice_weather_post_function, debugLogger, correlator)
workflow_engine = Component("workflow_engine", nice_weather_pre_function, nice_weather_post_function, debugLogger, analytics)
worker = Component("worker", nice_weather_pre_function, nice_weather_post_function, debugLogger, workflow_engine)
indicator_parser = Component("indicator_parser", nice_weather_pre_function, nice_weather_post_function, debugLogger, worker)
email_service = Component("email_service", nice_weather_pre_function, nice_weather_post_function, debugLogger, indicator_parser)
guardia_api = Component("guardia_api", nice_weather_pre_function, nice_weather_post_function, debugLogger, email_service)
auth_proxy = Component("auth_proxy", nice_weather_pre_function, nice_weather_post_function, debugLogger, guardia_api)

In [35]:
caseLogger = setup_logger("nice_weather_case")
case = CaseCapsule("1", caseLogger)

auth_proxy.exec(case)

DEBUG::auth_proxy::2021-01-05 11:37:38::DEBUG::before calling pre sub functions
DEBUG::auth_proxy::2021-01-05 11:37:38::DEBUG::executing pre function
CASE::id 1::2021-01-05 11:37:38.989062::count 2::DEBUG::executed pre function of component auth_proxy
DEBUG::auth_proxy::2021-01-05 11:37:38::DEBUG::after calling pre sub functions
DEBUG::auth_proxy::2021-01-05 11:37:38::DEBUG::before calling sub component
DEBUG::guardia_api::2021-01-05 11:37:38::DEBUG::before calling pre sub functions
DEBUG::guardia_api::2021-01-05 11:37:38::DEBUG::executing pre function
CASE::id 1::2021-01-05 11:37:39.089062::count 3::DEBUG::executed pre function of component guardia_api
DEBUG::guardia_api::2021-01-05 11:37:38::DEBUG::after calling pre sub functions
DEBUG::guardia_api::2021-01-05 11:37:38::DEBUG::before calling sub component
DEBUG::email_service::2021-01-05 11:37:38::DEBUG::before calling pre sub functions
DEBUG::email_service::2021-01-05 11:37:38::DEBUG::executing pre function
CASE::id 1::2021-01-05 11

## Simulating inconsistent and slow correlator

In [36]:
def long_pre_function(case:CaseCapsule, logger: logging.Logger, name):
    logger.debug('executing pre function')
    case.add_seconds(simulate_timeout(10, 0.4))
    case.log.debug(f'executed pre function of component {name}')

def long_post_function(case:CaseCapsule, logger: logging.Logger, name):
    logger.debug('executing post function')
    case.add_seconds(simulate_timeout(5, 0.3))
    case.log.debug(f'executed post function of component {name}')

notification_service = Component("notification_service", nice_weather_pre_function, nice_weather_post_function, debugLogger)
correlator = Component("correlator", long_pre_function, long_post_function, debugLogger, notification_service)
analytics = Component("analytics", nice_weather_pre_function, nice_weather_post_function, debugLogger, correlator)
workflow_engine = Component("workflow_engine", nice_weather_pre_function, nice_weather_post_function, debugLogger, analytics)
worker = Component("worker", nice_weather_pre_function, nice_weather_post_function, debugLogger, workflow_engine)
indicator_parser = Component("indicator_parser", nice_weather_pre_function, nice_weather_post_function, debugLogger, worker)
email_service = Component("email_service", nice_weather_pre_function, nice_weather_post_function, debugLogger, indicator_parser)
guardia_api = Component("guardia_api", nice_weather_pre_function, nice_weather_post_function, debugLogger, email_service)
auth_proxy = Component("auth_proxy", nice_weather_pre_function, nice_weather_post_function, debugLogger, guardia_api)

In [37]:
caseLogger = setup_logger("long_correlator_case")
case = CaseCapsule("1", caseLogger)

auth_proxy.exec(case)

DEBUG::auth_proxy::2021-01-05 11:37:44::DEBUG::before calling pre sub functions
DEBUG::auth_proxy::2021-01-05 11:37:44::DEBUG::executing pre function
CASE::id 1::2021-01-05 11:37:44.630360::count 2::DEBUG::executed pre function of component auth_proxy
DEBUG::auth_proxy::2021-01-05 11:37:44::DEBUG::after calling pre sub functions
DEBUG::auth_proxy::2021-01-05 11:37:44::DEBUG::before calling sub component
DEBUG::guardia_api::2021-01-05 11:37:44::DEBUG::before calling pre sub functions
DEBUG::guardia_api::2021-01-05 11:37:44::DEBUG::executing pre function
CASE::id 1::2021-01-05 11:37:44.730360::count 3::DEBUG::executed pre function of component guardia_api
DEBUG::guardia_api::2021-01-05 11:37:44::DEBUG::after calling pre sub functions
DEBUG::guardia_api::2021-01-05 11:37:44::DEBUG::before calling sub component
DEBUG::email_service::2021-01-05 11:37:44::DEBUG::before calling pre sub functions
DEBUG::email_service::2021-01-05 11:37:44::DEBUG::executing pre function
CASE::id 1::2021-01-05 11